In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import pandas as pd
from langchain_core.documents import Document
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

In [ ]:
doc = pd.read_csv('..\Title18_CSV_Data\Title18_processed_sections.csv', encoding='utf-8')
processed_content = doc['Processed_Content']

In [ ]:
doc.columns

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="nlpaueb/legal-bert-base-uncased")

In [14]:
result = embedding_model.embed_query(text=processed_content[67])

In [15]:
documents = [Document(page_content=row['Processed_Content'], metadata={'Section': row['Processed_Section'], 'Metadata': row['Processed_Metadata'],'Url': row['Url']}) for i, row in doc.iterrows()]

In [ ]:
embeddings = embedding_model.embed_documents([doc.page_content for doc in documents])

In [19]:
data = {
    'Processed_Content': [doc.page_content for doc in documents],
    'Embedding': embeddings,  # Assuming embeddings were created in the same order
    'Processed_Section': [doc.metadata['Section'] for doc in documents],
    'Processed_Metadata': [doc.metadata['Metadata'] for doc in documents],
    'Url': [doc.metadata['Url'] for doc in documents]
}

# Convert to DataFrame
df_embeddings = pd.DataFrame(data)

In [20]:
df_embeddings.to_csv('embeddings_nlpaueb_lbbu.csv', index=False)

In [ ]:

# Generate query embedding
query_embedding = embedding_model.embed_query(text=processed_content[67])

# Convert to NumPy arrays
query_embedding = np.array(query_embedding).reshape(1, -1)
embeddings = np.array(embeddings)

# Compute cosine similarity
similarities = cosine_similarity(query_embedding, embeddings)

# Get the index of the most similar document
most_similar_index = np.argmax(similarities)

print("Most similar document index:", most_similar_index)


Most similar document index: 67
